# Data Journalism Lesson 20: Scatterplots

Comparing two numbers to uncover a trend.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('plotly')
await micropip.install("nbformat>=4.2.0")

In [ ]:
from IPython.display import display, HTML
import pandas as pd

# --- Simple Grading/Checking Functions ---
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_df_exists(df, df_name, expected_min_rows=None, expected_cols=None):
    if not isinstance(df, pd.DataFrame) or df.empty:
        display_feedback(False, f'{df_name} DataFrame is not loaded correctly or is empty.', 'Please check the loading process.')
        return False
    msg_correct = f'{df_name} DataFrame loaded successfully.'
    correct = True
    msg_incorrect_list = []

    if expected_min_rows is not None and len(df) < expected_min_rows:
        msg_incorrect_list.append(f' Expected at least {expected_min_rows} rows, got {len(df)}.')
        correct = False
    if expected_cols is not None:
      if not all(col in df.columns for col in expected_cols):
        missing_cols = [col for col in expected_cols if col not in df.columns]
        msg_incorrect_list.append(f' Missing expected columns: {missing_cols}.')
        correct = False
    
    if correct:
        display_feedback(True, msg_correct, '')
    else:
        display_feedback(False, '', ' '.join(msg_incorrect_list))
    return correct

def check_plot_params(params, expected_params, plot_name):
    correct = True
    messages = []
    for p_name, p_val_expected in expected_params.items():
        p_val_actual = params.get(p_name)
        if isinstance(p_val_actual, pd.DataFrame) and isinstance(p_val_expected, pd.DataFrame):
            # For DataFrames, a simple check for column existence might be enough for exercises
            if all(col in p_val_actual.columns for col in p_val_expected.columns):
                 messages.append(f'Correct DataFrame structure for {p_name} in {plot_name}.')
            else:
                 correct = False
                 messages.append(f'DataFrame for {p_name} in {plot_name} missing expected columns.')
        elif p_val_actual == p_val_expected:
            messages.append(f'Correct {p_name} for {plot_name}.')
        else:
            correct = False
            messages.append(f'Incorrect {p_name} for {plot_name}. Expected \'{p_val_expected}\', got \'{p_val_actual}\'.')
    final_message_correct = f'Plot parameters for {plot_name} are correct!'
    final_message_incorrect = ' '.join(messages)
    display_feedback(correct, final_message_correct, final_message_incorrect)

In [ ]:
# --- State Setup and Data Loading ---
state_abbr = 'MN'
state_full_name = 'Minnesota'


colleges_national_url = "../_static/college-cost/national.csv"

colleges_df_initial = pd.read_csv(colleges_national_url)
state_colleges_df_initial = colleges_df_initial[colleges_df_initial['STABBR'] == state_abbr].copy()
state_colleges_count_expected = len(state_colleges_df_initial)

# Pluralization for glue text
s_char = 's' if state_colleges_count_expected != 1 else ''
is_are = 'are' if state_colleges_count_expected != 1 else 'is'
college_plural_str1 = f"There {is_are} {state_colleges_count_expected} college{s_char} in the state."
college_plural_str2 = f"{state_colleges_count_expected} institution{s_char} that grant bachelor's degrees."


In [ ]:
from myst_nb import glue

glue("state_full_name", state_full_name, display=False)
glue("state_abbr", state_abbr, display=False)
glue("state_colleges_count", state_colleges_count_expected, display=False)
glue("collegeplural", college_plural_str1, display=False)
glue("collegeplural2", college_plural_str2, display=False)

## The Goal

In this lesson, you'll learn how to create and interpret scatterplots, a powerful tool for visualizing relationships between two numerical variables. By the end of this tutorial, you'll understand how to prepare data for scatterplots, construct basic plots using Plotly Express, highlight specific data points, and add labels. You'll practice these skills using real-world college completion and cost data, gaining insight into how to uncover and communicate trends and outliers in your datasets. These abilities will be crucial for identifying patterns and relationships in your data journalism projects.

## Why Visualize Data?

One of the more interesting thinkers about data visualization on planet earth today is Alberto Cairo. He is the Knight chair in visual journalism at the School of Communication at the University of Miami. He's written four books in English about data visualization -- *The Functional Art* (2013), *The Truthful Art* (2016), *How Charts Lie* (2019) and *The Art of Insight* (2023). If you're serious about data visualization, you own these books and if he is speaking somewhere near you, you go and you listen. 

I was hooked early by Cairo's vision for data visualization -- and the kind of person who does the work -- from the first chapter of *The Functional Art*. Specifically, he wrote that "the life of a visual communicator should be one of systematic and exciting intellectual chaos." I felt like my own read-anything-that-catches-my-attention lifestyle was all okay now. Intellectual curiosity -- spread far and wide -- leads to inspiration, which leads to all kinds of interesting things. 

For me -- your humble author -- Tufte and Cairo are two voices in my head when I am trying to make something out of data. They're fighting for the soul of my idea. On the one side, there is comfort in rules that simplify and dictate forms and functions. On the other is Cairo, talking about art and creativity and how rules can squash something great into something mediocre. 

What do I mean by this? And what do I mean by Cairo being one of the most fascinating thinkers about data visualization?

In August 2023, Cairo made an appearance on the Real World Data Science podcast to talk about his not-yet-released book *The Art of Insight*. In it, he talked about how visualization is like language in that it has dialects. The dialects of data viz are the places that use them -- journalism, science, industry, art, and others. The rules for how we write in journalism, for example, are vastly different from how we write in the sciences. So why are we trying to create rigid rules for all data visualization? 

"There are really no universal rules in data visualization: they are parochial rules that are applicable to the different dialects," Cairo said. "But it is wrong -- it is a mistake -- to apply the standards of one of the dialects of data visualization to a completely different dialect of data visualization." 

And, not just that, the "rules" are kind of a mishmash of empirical research and opinions. 

"There are a few things that could be considered rules," he said. "For example we know that if you want to compare numbers a bar graph is usually superior to a pie chart, for example. We know that there's empirical evidence behind that so you can sort of like derive a principle out of that, right, but beyond those very basic things there are really not many rules."

What do we have? Guidelines. Inherited wisdom. Historical examples. And you should learn them, Cairo said, along with the perception and cognition research that can guide us. 

"But after you do that all that matters is the choices that you make with the knowledge that you have and with the guesses that you can make," he said.

Today we're working with what I believe is the most versatile and interpretive graph type around -- the scatterplot. What makes the scatterplot so great? You use it to compare two dimensions of data -- so your x and y are both numbers. That means the space and the arrangement of dots in it is meaningful. If they cluster together? That means something. If they arrange along a path going up and to the right? Meaningful. Down and right? Meaningful. You can even turn the space into quadrants -- above and below average on your two dimensions of data -- and where dots appear means something. And, unlike many other graphical types, scatterplots are better, in my opinion, when there is *more* data. A scatterplot with just a few points isn't a very compelling scatterplot. A scatterplot with *thousands* can be.

## The Basics

One of the most powerful things that we can do with visualizing data is establish relationships and find when things don't fit the relationship. 

In a previous assignment, we looked at colleges. Next question: Is there a relationship between how much it costs to go to a college and how many people graduate? In other words -- if it's really expensive to go there, is that creating problems for people finishing their degrees? Or is it that the more expensive the school, the more likely those people graduate? Maybe they only let smart people in. Maybe they only let rich people in who don't worry about the cost. Maybe all of those things. We don't know from just the question, but a scatterplot can help us explore this.

We'll start with libraries. Plotly Express (`px`) will be our primary tool.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

We're going to work with a slightly different version of the college data. In this case, the differences are: 

* We're only looking at four-year colleges, ones that primarily grant bachelor's and graduate degrees, for the entire nation.
* The two fields we're going to look at are the average cost of attendance and the completion rate. The average cost of attendance -- `COSTT4_A` in the data -- is the total cost for a year at that school, including tuition, housing, books and fees. The completion rate -- `C150_4` in the data -- is the percent of first-time, first-semester freshmen who complete their degree in *6* years.
* I have filtered out schools who didn't report one or both of these bits of data. Example: There are 23 schools who reported zero students graduating in 6 years. That's ... odd, and likely a data quality issue or specific circumstance not suitable for this general analysis.

Let's import the data now.

In [ ]:
colleges_df = pd.read_csv("../_static/college-cost/national.csv")

Scatterplots are, for my money, the most revealing and useful chart type out there that isn't a map. They reveal so much, and they allow you to make comparisons between two numbers. With a scatterplot, we put what we think predicts the thing on the x-axis, and the thing being predicted on the y-axis. `px.scatter()` works just like other Plotly Express functions -- we need `x` and `y` arguments and a `data_frame`. Both x and y are numbers. 

The other advantage of scatterplots? You can put a *huge* amount of data on them. There are more than 1,700 colleges that fit our criteria. Do we need to narrow it down for us to fit them all in the chart? Nope.

Let's take a look at our data so we know what we can work with going forward.

In [ ]:
display(____.head())

### Exercise 1: Making a scatterplot

Remember -- we're looking at how much it costs to go to that school (`COSTT4_A`) and comparing it to the completion rate (`C150_4`). Create a scatterplot using these columns. For now, set all points to a 'grey' color by updating the traces after creating the figure.

In [ ]:
fig_ex1 = px.scatter(
    data_frame=____,
    x=____, # Average Cost of Attendance
    y=____, # Completion Rate
    title='College Cost vs. Completion Rate (National)'
)

# Set all points to grey
fig_ex1.update_traces(marker=dict(color=____)) # e.g., 'lightgrey'

fig_ex1.show()

The way to read this chart -- when you're looking for relationships -- is do the dots sort of cluster in a pattern? In this case ... they do! At the lower costs, they all kind of bunch together with a wide range of completion rates, but as the cost gets higher, the completion percentage generally tends to get higher too. See how they flow up and to the right? That's the relationship. The more expensive the school (generally), the more likely a student is to complete their degree there. A massive question that should be rattling around in your head right now: Why?

Where are {glue:text}`state_full_name` colleges on this chart? There are {glue:text}`state_colleges_count` colleges in the state. You may have noticed we set the others to grey. Let's make these a color. But first, we have to make a DataFrame of them.

### Exercise 2: Colleges in your state 

The way to highlight specific dots (e.g., for one state) is to add another layer or trace to our plot. First, the dataframe. We’ll call it `state_colleges_df`, and the state abbreviation you want is your state’s two-letter postal code.

In [ ]:
state_colleges_df = colleges_df[colleges_df['STABBR'] == "_____"]
display(state_colleges_df.head())

Now we have a DataFrame of {glue:text}`state_colleges_count` institutions. Let's use it in a new trace.

### Exercise 3: Red dots please

Add a new trace to the figure from Exercise 1 (`fig_ex1`) for `state_colleges_df`. Set the color of these new points to 'red'. Use `fig.add_trace()` and provide a `go.Scatter` object.

In [ ]:
fig_ex3 = px.scatter( # Recreate the base for clarity, or use go.Figure(fig_ex1)
    data_frame=colleges_df,
    x='COSTT4_A',
    y='C150_4',
    title=f'Cost vs. Completion: {state_full_name} Highlighted'
)
fig_ex3.update_traces(marker=dict(color='lightgrey', size=5), selector=dict(name=fig_ex3.data[0].name))

fig_ex3.add_trace(go.Scatter(
    x=state_colleges_df[____], # Cost
    y=state_colleges_df[____], # Completion
    mode='markers',
    marker=dict(color=____, size=7), # e.g., 'red'
    name=state_full_name # For legend
))

fig_ex3.show()

And there's a chart of {glue:text}`state_full_name`'s {glue:text}`state_colleges_count` institutions that grant bachelor's degrees. But who is who?

To get labels on there, we're going to modify the trace we added for the highlighted state. We can add a `text` property to `go.Scatter` (or `px.scatter`) which contains the labels (e.g., `INSTNM`), and set `mode='markers+text'`. We can also control text position.

For very dense plots, some labels might overlap.

### Exercise 4: Labeling

Recreate the plot from Exercise 3. This time, when you add the trace for `state_colleges_df`:
1.  Set `mode='markers+text'`.
2.  Pass the `INSTNM` column to the `text` argument.
3.  Optionally, set `textposition='top center'` or another value to position the text relative to the markers.

In [ ]:
fig_ex4 = px.scatter(
    data_frame=colleges_df,
    x='COSTT4_A',
    y='C150_4',
    title=f'Cost vs. Completion: {state_full_name} Colleges Labeled'
)
fig_ex4.update_traces(marker=dict(color='lightgrey', size=5), selector=dict(name=fig_ex4.data[0].name))

fig_ex4.add_trace(go.Scatter(
    x=state_colleges_df['COSTT4_A'], 
    y=state_colleges_df['C150_4'], 
    mode=____,
    text=state_colleges_df[____],
    textposition=____, # e.g., 'top center', 'middle right'
    textfont=dict(size=9, color='darkblue'),
    marker=dict(color='red', size=7),
    name=state_full_name
))
fig_ex4.update_layout(height=max(600, state_colleges_count_expected * 20)) # Adjust height if many labels
fig_ex4.show()

And now you know which and where {glue:text}`state_full_name`'s colleges reside in this Cheap/Expensive vs. Graduates Nobody/Everybody comparison.

But let's be honest: it might be a bit of a mess if there are many labels. Do you even see all of them? Do you need that many labels on there? What you would normally do at this point is pick and choose who you label. Is it just the largest? Is it a specific set of schools? You probably don't want more than about five to seven labels on a dense scatterplot to avoid cluttering it up too much, unless the interactivity of Plotly (like hover labels) is sufficient.

## The Recap

Throughout this lesson, you've mastered the basics of creating informative scatterplots with Plotly Express and Plotly Graph Objects. You've learned how to plot multiple data points to reveal overall trends, highlight specific points of interest by adding and styling traces, and add text labels to key data points. Remember, scatterplots are excellent for visualizing relationships between two numerical variables and identifying outliers. As you gain more experience, you'll find scatterplots invaluable for uncovering stories within your data, such as the relationship between college costs and completion. Always consider how you can use color, labels, and additional data points to enhance the story your scatterplot tells, and leverage Plotly's interactivity (like tooltips on hover) to provide more detail without cluttering the main visual.

## Terms to Know

- **Scatterplot**: A type of chart that shows the relationship between two numerical variables, with each data point represented as a dot on a two-dimensional plane.
- **`plotly.express.scatter()` (or `px.scatter()`)**: A Plotly Express function used to create scatterplots.
- **`plotly.graph_objects.Scatter` (or `go.Scatter`)**: A trace type in Plotly used for creating scatter and line plots, offering detailed control. Used here for adding highlighted points with labels.
- **`x`, `y` arguments**: In `px.scatter` or `go.Scatter`, these map data to the x and y axes.
- **`marker=dict(color=...)`**: Used to set properties of markers (points) in Plotly traces, such as color and size.
- **`text` argument**: In `px.scatter` or `go.Scatter`, used to provide text labels for data points.
- **`mode='markers+text'`**: A mode for `go.Scatter` traces to display both markers and text labels.
- **`textposition`**: An argument to control the position of text labels relative to the markers (e.g., 'top center', 'middle right').
- **`fig.add_trace()`**: A Plotly method to add a new trace (like a set of scatter points) to an existing figure.
- **`fig.update_traces()`**: A Plotly method to modify properties of existing traces in a figure.